In [1]:
# import pacakges
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns',None)

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

import pickle

# Import Data

In [2]:
X_train = pd.read_csv('data/train-test/train_nulls_dropped.csv')
X_test = pd.read_csv('data/train-test/test_nulls_dropped.csv')

In [3]:
features = ['Pitcher_name','batter_name','pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up']
features2 = ['batter_name','pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up']
features3 = ['pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up']

In [4]:
X_train = X_train[features3]
X_test = X_test[features3]

In [16]:
X_train.head()

,release_speed,release_pos_x,release_pos_z,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,inning,inning_topbot,effective_speed,release_spin_rate,pitch_number,bat_score,fld_score,vx0,vy0,vz0,ax,ay,az,outs_when_up,pitch_name_4-Seam Fastball,pitch_name_Changeup,pitch_name_Curveball,pitch_name_Cutter,pitch_name_Forkball,pitch_name_Knuckle Curve,pitch_name_Knuckleball,pitch_name_Sinker,pitch_name_Slider,pitch_name_Split-Finger,pitch_name_Unknown
0,96.1,-1.92,5.95,1,1,1,2,-0.27,1.70,-0.93,1.62,12,0,95.4,2463.0,6,2,7,3.075,-139.609,-9.724,-2.028,34.780,-11.706,2,1,0,0,0,0,0,0,0,0,0,0
1,93.1,-1.50,6.02,1,1,1,2,0.24,0.72,0.78,1.55,12,0,92.2,2705.0,5,2,7,5.156,-135.442,-7.227,5.004,28.835,-25.323,2,0,0,0,1,0,0,0,0,0,0,0
2,97.0,-1.37,6.08,1,1,1,2,-0.50,1.41,1.32,2.47,12,0,96.4,2362.0,4,2,7,8.459,-140.843,-7.313,-5.412,31.551,-15.189,2,1,0,0,0,0,0,0,0,0,0,0
3,93.6,-1.25,6.06,1,1,0,2,0.32,0.80,2.92,2.23,12,0,92.8,2724.0,3,2,7,10.094,-135.734,-5.821,6.079,26.355,-24.250,2,0,0,0,1,0,0,0,0,0,0,0
4,97.1,-1.49,6.08,1,1,0,1,-0.76,1.63,0.26,2.74,12,0,96.0,2401.0,2,2,7,6.519,-140.993,-7.138,-9.148,32.251,-11.956,2,1,0,0,0,0,0,0,0,0,0,0


In [32]:
X_train.columns[:27]

Index(['pitch_name', 'release_speed', 'release_pos_x', 'release_pos_z',
       'stand', 'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x',
       'plate_z', 'inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0',
       'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up'],
      dtype='object')

In [5]:
# We will pd.get_dummies 
X_train = pd.get_dummies(X_train, columns=['pitch_name'])
X_test = pd.get_dummies(X_test, columns=['pitch_name'])

In [6]:
# drop columns that don't match in each other
drop_test = [col for col in X_test.columns if col not in X_train.columns]
drop_train = [col for col in X_train.columns if col not in X_test.columns]

X_train.drop(columns=drop_train, inplace=True)
X_test.drop(columns=drop_test, inplace=True)

In [7]:
X_train.columns == X_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [8]:
y_train = pickle.load(open('assets/y_train.pkl','rb'))
y_test = pickle.load(open('assets/y_test.pkl','rb'))

In [10]:
y_train.head()

0    strike
1    strike
2    strike
3      ball
4    strike
Name: strike_attempt, dtype: object

# Set up Pipeline/GridSearchCV

---
Unable to do GridSearch because I have been running into too many memory errors

In [11]:
# list(range(1, 101, 10))

In [12]:
# rf_params = {
#     'n_neighbors': list(range(1, 101, 10)),
#     'min_samples_split':[2,3],
#     'max_features':["auto",28],
#     'max_samples':[.3,.5,.7]
# } 

In [13]:
# rf = GridSearchCV(RandomForestClassifier(random_state=66),
#                   param_grid=rf_params,
#                   cv=5,
#                   n_jobs=8,
#                   verbose = 2)

In [14]:
# rf.fit(X_train,y_train)

In [15]:
rf = RandomForestClassifier(random_state=66, n_jobs=8)
rf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=8, random_state=66)

In [17]:
preds = rf.predict(X_test)

In [18]:
accuracy_score(y_test, preds)

0.7092507858764223

In [21]:
df = pd.DataFrame(rf.feature_importances_, index=X_train.columns)

In [27]:
df.sort_values(0, ascending=False).head(35)

,0
plate_x,0.137504
plate_z,0.115888
vz0,0.064031
vx0,0.051252
release_pos_x,0.038365
az,0.037864
release_pos_z,0.036782
ax,0.036452
ay,0.035946
release_spin_rate,0.035828


In [28]:
pickle.dump(rf, open('assets/rf_no_pitcher_12062020.pkl','wb'))

# Random Forest model, no pitchers and batters

In [9]:
# rf_params = {
# #     'n_neighbors': list(range(1, 101, 10)),
#     'min_samples_split':[2,3],
# #     'min_samples_leaf':[1,2,3],
#     'max_features':['auto','log2'],
#     'max_samples':[.95,.99]
# } 
# rf2 = GridSearchCV(RandomForestClassifier(random_state=66),
#                   param_grid=rf_params,
#                   cv=5,
#                   n_jobs=8,
#                   verbose = 2)

In [9]:
rf2 = RandomForestClassifier(random_state=66, n_jobs=8,verbose=2)

In [10]:
rf2.fit(X_train, y_train)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100

building tree 4 of 100
building tree 5 of 100
building tree 6 of 100building tree 7 of 100


building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100building tree 21 of 100

building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  2.0min


building tree 33 of 100building tree 34 of 100

building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:  6.5min finished


RandomForestClassifier(n_jobs=8, random_state=66, verbose=2)

In [11]:
preds2 = rf2.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    3.7s finished


In [12]:
accuracy_score(y_test, preds2)

0.7113739053224779

In [14]:
df2 = pd.DataFrame(rf2.feature_importances_, index=X_train.columns)

In [15]:
df2.sort_values(0, ascending=False).head(35)

,0
plate_x,0.182280
plate_z,0.147995
vz0,0.064584
vx0,0.051666
az,0.044295
release_pos_x,0.043263
release_pos_z,0.041851
ax,0.041754
release_spin_rate,0.041669
ay,0.041623
